In [14]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, MT5ForConditionalGeneration, MT5Tokenizer
import pandas as pd
import numpy as np
import os
import re
import xml.etree.ElementTree as ET
from pathlib import Path
from pandarallel import pandarallel

In [25]:
def test_to_df(path):
    pathlist = Path(path).glob('**/*.xml')
    ids=[]
    x_raw=[]
    for path in pathlist:  
        head, tail = os.path.split(path)
        t=tail.split(".")
        path_in_str = str(path)
        tree = ET.parse(path_in_str)
        root = tree.getroot()
        for child in root:
            xi=[]
            for ch in child:
                ids.append(t[0])
                x_raw.append(ch.text)

    test_df=pd.DataFrame()
    test_df["id"]=ids
    test_df["tweets"]=x_raw
    return test_df

In [26]:
esdf = test_to_df('es-test')
endf = test_to_df('en-test')

In [27]:
SYMBOLS_TO_ISOLATE = '.,?!-;*"…:—()%#$&_/@＼・ω+=”“[]^–>\\°<~•≠™ʊɒ∞§{}·τα❤☺ɡ|¢→̶`❥━┣┫┗Ｏ►★©―ɪ✔®\x96\x92●£♥➤´¹☕≈÷♡◐║▬′ɔː€۩۞†μ✒➥═☆ˌ◄½ʻπδηλσερνʃ✬ＳＵＰＥＲＩＴ☻±♍µº¾✓◾؟．⬅℅»Вав❣⋅¿¬♫ＣＭβ█▓▒░⇒⭐›¡₂₃❧▰▔◞▀▂▃▄▅▆▇↙γ̄″☹➡«φ⅓„✋：¥̲̅́∙‛◇✏▷❓❗¶˚˙）сиʿ✨。ɑ\x80◕！％¯−ﬂﬁ₁²ʌ¼⁴⁄₄⌠♭✘╪▶☭✭♪☔☠♂☃☎✈✌✰❆☙○‣⚓年∎ℒ▪▙☏⅛ｃａｓǀ℮¸ｗ‚∼‖ℳ❄←☼⋆ʒ⊂、⅔¨͡๏⚾⚽Φ×θ￦？（℃⏩☮⚠月✊❌⭕▸■⇌☐☑⚡☄ǫ╭∩╮，例＞ʕɐ̣Δ₀✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒↑☝ɹ✅☛♩☞ＡＪＢ◔◡↓♀⬆̱ℏ\x91⠀ˤ╚↺⇤∏✾◦♬³の｜／∵∴√Ω¤☜▲↳▫‿⬇✧ｏｖｍ－２０８＇‰≤∕ˆ⚜☁'
SYMBOLS_TO_REMOVE = '\n\r\xa0\ue014\t\uf818\uf04a\xad\uf0e0\u200b\u200eعدويهصقأناخلىبمغرЕ\u202a\u202c🏻ᴍʏʀᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢשלוםבי‼\x81エンジ故障\u2009ᴵ͞\u200fאעכחஜᴠ‐\x7fἐὶήιὲκἀίῃἴξＨ\ufeff\u2028\u3000تحكسة👮💙فزط\u2008🏾\x08‑地獄谷улкнПоАН歌舞伎הмυтѕ⤵\u200aэпрд\x95\u2002\u2004ˢᵒʳʸᴼᴷᴺʷᵗʰᵉᵘ\x13\ue602άοόςέὸתמדףנרךצט\uf0b7\uf04c\x9f\x10成都\u202d💤🍇\ue613小土豆🏡❔⁉\u202f👠》कर्मा🇹🇼🌸蔡英文🌞🎲レクサス外国人关系Сб💋💀🎄💜ьыгя不是\x9c\x9d🗑\u2005💃📣༼つ༽ḷЗз▱ц￼卖温哥华议会下降你失去所有的钱加拿大坏税骗子🐝ツ🎅\x85🍺آإشء🎵🌎͟ἔ油别克🤡й\u2003🚀🤴ʲшчИОРФДЯМюж🖑ὐύύ特殊作戦群щ💨圆明园קℐ\u200d𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪天一家⚲\u2006⚭⚆⬭⬯⏖新✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺ϖ\u2000үսᴦᎥһͺ\u2007հ\u2001ɩｙｅ൦ｌƽｈ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧Ƅᴨןᑯ໐ΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓ё𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘ͦ💸ج패티Ｗ𝙇ᵻ👂👃ɜ🎫\uf0a7БУі🚢🚂ગુજરાતીῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼☘﴾̯﴿₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸公寓养宠物吗🏄🐀🚑🤷操美𝒑𝒚𝒐𝑴🤙🐒欢迎来到阿拉斯ספ𝙫🐈𝒌𝙊𝙭𝙆𝙋𝙍𝘼𝙅ﷻ🦄巨收赢得白鬼愤怒要买额ẽ🚗🐳𝟏𝐟𝟖𝟑𝟕𝒄𝟗𝐠𝙄𝙃👇锟斤拷𝗢𝟳𝟱𝟬⦁マルハニチロ株式社⛷한국어ㄸㅓ니͜ʖ𝘿𝙔₵𝒩ℯ𝒾𝓁𝒶𝓉𝓇𝓊𝓃𝓈𝓅ℴ𝒻𝒽𝓀𝓌𝒸𝓎𝙏ζ𝙟𝘃𝗺𝟮𝟭𝟯𝟲👋🦊多伦🐽🎻🎹⛓🏹🍷🦆为和中友谊祝贺与其想象对法如直接问用自己猜本传教士没积唯认识基督徒曾经让相信耶稣复活死怪他但当们聊些政治题时候战胜因圣把全堂结婚孩恐惧且栗谓这样还♾🎸🤕🤒⛑🎁批判检讨🏝🦁🙋😶쥐스탱트뤼도석유가격인상이경제황을렵게만들지않록잘관리해야합다캐나에서대마초와화약금의품런성분갈때는반드시허된사용🔫👁凸ὰ💲🗯𝙈Ἄ𝒇𝒈𝒘𝒃𝑬𝑶𝕾𝖙𝖗𝖆𝖎𝖌𝖍𝖕𝖊𝖔𝖑𝖉𝖓𝖐𝖜𝖞𝖚𝖇𝕿𝖘𝖄𝖛𝖒𝖋𝖂𝕴𝖟𝖈𝕸👑🚿💡知彼百\uf005𝙀𝒛𝑲𝑳𝑾𝒋𝟒😦𝙒𝘾𝘽🏐𝘩𝘨ὼṑ𝑱𝑹𝑫𝑵𝑪🇰🇵👾ᓇᒧᔭᐃᐧᐦᑳᐨᓃᓂᑲᐸᑭᑎᓀᐣ🐄🎈🔨🐎🤞🐸💟🎰🌝🛳点击查版🍭𝑥𝑦𝑧ＮＧ👣\uf020っ🏉ф💭🎥Ξ🐴👨🤳🦍\x0b🍩𝑯𝒒𝟐🏂👳🍗🕉🐲چی𝑮𝗕𝗴🍒ꜥⲣⲏ🐑⏰鉄リ事件ї💊「」\uf203\uf09a\uf222\ue608\uf202\uf099\uf469\ue607\uf410\ue600燻製シ虚偽屁理屈Г𝑩𝑰𝒀𝑺🌤𝗳𝗜𝗙𝗦𝗧🍊ὺἈἡχῖΛ⤏🇳𝒙ψՁմեռայինրւդձ冬至ὀ𝒁🔹🤚🍎𝑷🐂💅𝘬𝘱𝘸𝘷𝘐𝘭𝘓𝘖𝘹𝘲𝘫کΒώ💢ΜΟΝΑΕ🇱♲𝝈↴💒⊘Ȼ🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻𝐎𝐍𝐊𝑭🤖🎎😼🕷ｇｒｎｔｉｄｕｆｂｋ𝟰🇴🇭🇻🇲𝗞𝗭𝗘𝗤👼📉🍟🍦🌈🔭《🐊🐍\uf10aლڡ🐦\U0001f92f\U0001f92a🐡💳ἱ🙇𝗸𝗟𝗠𝗷🥜さようなら🔼'
POSITIVE_EMOJI = '😜😎😁💖😀😂😄😋👏😊👍😃😘👌🙂😉😏🎉😅👻🙃😆🤗🤓😌🤑😛🤣😝💪😗🥰😇🤠🤡🥳🥴🤩😺😸😹😻😽✌️🤟🤘'
NEGATIVE_EMOJI = '🥵😢👎😱😳😧🙀😐😕😮😖😟😡😠😤😞😭😥😔😓😪😨😩🙁😵😒͝😣😲😯🤢َِ😰👿👿🤥😬😷🤒🤕🤯🤬🥺🙀😿😾🖕🏻🖕🏼'
FLAG_EMOJI = '🇯🇵🇰🇷🇩🇪🇨🇳🇺🇸🇫🇷🇪🇸🇮🇹🇲🇽🇮🇳🇷🇺🇬🇧'
NEUTRAL_EMOJI = '🎤🐶️🍕🐵💵🔥💥🚌🌟💩💯⛽🚄🏼🚲😈🙏🎯🌹💔👊🙄⛺🍾🏽🎆🍻⏺🌏💞🚓🔔📚🏀👐🍽🎶🌺🤔🐰🐇🏈😺🌍🍔🐮🍁🍆🍑🌮🌯🤦🙈😴🆕👅👥👄🔄🔤👉👤🤧👶👲🔛🎓🏿🇺🇸🌠🐟💫💰🚬💎🐱🙆💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚🐾🐕🔗🚽🏆🎃🖐🙅⛲🍰🤐👆🙌💛👀🙊🙉'
ISOLATE_DICT = {ord(c):''.format(ord(c)) for c in SYMBOLS_TO_ISOLATE}
REMOVE_DICT = {ord(c):'' for c in SYMBOLS_TO_REMOVE}
FLAG_EMOJI_DICT = {ord(c):'' for c in FLAG_EMOJI}
NEUTRAL_EMOJI_DICT = {ord(c):'' for c in NEUTRAL_EMOJI}
POSITIVE_EMOJI_DICT = {ord(c):'' for c in POSITIVE_EMOJI}
NEGATIVE_EMOJI_DICT = {ord(c):'' for c in NEGATIVE_EMOJI}
CONTRACTION_MAPPING = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }


def handle_punctuation(text):
    text = text.translate(REMOVE_DICT)
    text = text.translate(NEUTRAL_EMOJI_DICT)
    text = text.translate(POSITIVE_EMOJI_DICT)
    text = text.translate(FLAG_EMOJI)
    text = text.translate(NEGATIVE_EMOJI_DICT)
    text = text.translate(ISOLATE_DICT)
    return text

def clean_contractions(text, mapping=CONTRACTION_MAPPING):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text


def text_general_reworking(text):
    spaces = ['\u200b', '\u200e', '\u202a', '\u202c', '\ufeff', '\uf0d8', '\u2061', '\x10', '\x7f', '\x9d', '\xad',
              '\xa0']
    for space in spaces:
        text = text.replace(space, ' ')
    text = re.sub('\s+', ' ', text)
    text = re.sub(r'\.{2,}', ' ', text)
    text = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', '', text)
    text = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', '', text)
    text = re.sub(r'(<3|:\*)', '', text)
    text = re.sub(r'(;-?\)|;-?D|\(-?;)', '', text)
    text = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', '', text)
    text = re.sub(r'(:,\(|:\'\(|:"\()', '', text)
    #text = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', '', text)
    text = re.sub(r'\brt\b', '', text)
    text = re.sub(r'\[math\]', ' LaTex math ', text)
    text = re.sub(r'\[\/math\]', ' LaTex math ', text)
    text = re.sub(r'\\', ' LaTex ', text)
    text = re.sub(r'\brt\b', '', text)
    text = re.sub("([^\"^\'].\s)(\")([A-Z,a-z?])", r"\1\3", text)
    text = re.sub("(\')(.\")", r"\2", text)
    text = text.lower()
    if '-' in text:
        text = text.replace('-', ' - ')
    text.strip()
    return text

def en_preprocess(x):
    x = text_general_reworking(x)
    x = handle_punctuation(x)
    x = clean_contractions(x)
    return x

def es_preprocess(x):
    x = text_general_reworking(x)
    x = handle_punctuation(x)
    return x

In [28]:
pandarallel.initialize()
endf['input'] = endf['tweets'].parallel_apply(en_preprocess)
esdf['input'] = esdf['tweets'].parallel_apply(es_preprocess)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [31]:
endf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20000 non-null  object
 1   tweets  20000 non-null  object
 2   input   20000 non-null  object
dtypes: object(3)
memory usage: 468.9+ KB


In [35]:
endf['output'] = ""
esdf['output'] = ""
endf['output_alt'] = ""

In [36]:
def full_predict(df, start, stop, step, model, tokenizer, column):
    for i in range(start, stop):
        tweet = "hater classification: "+ df.iloc[i,2]
        input_ids = tokenizer.encode(tweet, return_tensors = "pt")
        output_ids = model.generate(input_ids)
        output = str(tokenizer.decode(output_ids[0], skip_special_tokens=True))
        df.iloc[i,column]=output
        if i%step==0:
            print("{} % complete".format(i/(step/10)))
    return df

In [34]:
multi_model = MT5ForConditionalGeneration.from_pretrained("PaulAdversarial/PAN_twitter_hate_speech_2021_ES_MT5")
multi_tokenizer = MT5Tokenizer.from_pretrained("PaulAdversarial/PAN_twitter_hate_speech_2021_ES_MT5")

In [37]:
esdf = full_predict(esdf, 0, 20000, 2000, multi_model, multi_tokenizer, 3)

0.0 % complete
10.0 % complete
20.0 % complete
30.0 % complete
40.0 % complete
50.0 % complete
60.0 % complete
70.0 % complete
80.0 % complete
90.0 % complete


In [38]:
def replace_bin(x):
    if x=='neutral':
        x=0
    elif(x=='hatespeach'):
        x=1
    else:
        print(x)
        x=0
    return int(x)

In [39]:
esdf['output_num'] = esdf['output'].parallel_apply(replace_bin)

In [40]:
esdf.id.value_counts()

69d556839af1b8b0756f1619a0178247    200
e8a2fed867e38a6ffb404e96221016cf    200
0389ca1de412788fe6e2ad1aff9a37b2    200
18172d25b495577ee01fdf2101dcb3cd    200
78dc139860cb2a5258967a46f6150d65    200
                                   ... 
7b5c34c0d03cf69dd4a4c1d753d5a7e6    200
b6e31de389da07af3d104bf991124b9a    200
c7deb138f9201ccc4c60293f99b6c929    200
5cd3e71439e942c65a65bc1d13a73dad    200
652ecfa77cdd304a12306da2130b4658    200
Name: id, Length: 100, dtype: int64

In [42]:
def assign_top(df,step=200):
    for i in range(0, int(df.shape[0]/step)):
        df.loc[i*step:(i+1)*step-1,'output_num_avg'] = int(round(df.loc[i*step:(i+1)*step-1,'output_num'].mean()))
    return df

In [43]:
esdf = assign_top(esdf)

In [66]:
esdf.output_num_avg = esdf.output_num_avg.astype(int)

In [67]:
esdf.to_csv("results_es/results.csv")

In [45]:
model = T5ForConditionalGeneration.from_pretrained("PaulAdversarial/T5_PAN_Hate_Speech_Twitter_topic_ishatespeach")
tokenizer = T5Tokenizer.from_pretrained("PaulAdversarial/T5_PAN_Hate_Speech_Twitter_topic_author_ishatespeach")

In [46]:
endf = full_predict(endf, 0, 20000, 2000, model, tokenizer, 3)

0.0 % complete
10.0 % complete
20.0 % complete
30.0 % complete
40.0 % complete
50.0 % complete
60.0 % complete
70.0 % complete
80.0 % complete
90.0 % complete


In [47]:
endf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          20000 non-null  object
 1   tweets      20000 non-null  object
 2   input       20000 non-null  object
 3   output      20000 non-null  object
 4   output_alt  20000 non-null  object
dtypes: object(5)
memory usage: 781.4+ KB


In [48]:
endf.head()

,id,tweets,input,output,output_alt
0,35dad41e25a45dcf6d7e3dd905dd0914,#USER# #HASHTAG# are the #HASHTAG#!,user hashtag are the hashtag,hatespeach,
1,35dad41e25a45dcf6d7e3dd905dd0914,#USER# Let's not pretend that you have a lick ...,user let us not pretend that you have a lick o...,hatespeach,
2,35dad41e25a45dcf6d7e3dd905dd0914,#USER# No #HASHTAG# for any of these #HASHTAG#...,user no hashtag for any of these hashtag hashtag,hatespeach,
3,35dad41e25a45dcf6d7e3dd905dd0914,Because you are against it - moving #HASHTAG# ...,because you are against it moving hashtag i...,neutral,
4,35dad41e25a45dcf6d7e3dd905dd0914,When is #HASHTAG# going to #HASHTAG#? #HASHTAG...,when is hashtag going to hashtag hashtag hasht...,neutral,


In [49]:
model_alt = T5ForConditionalGeneration.from_pretrained("PaulAdversarial/T5_PAN_Hate_Speech_Twitter_topic_author_ishatespeach")

In [50]:
endf = full_predict(endf, 0, 20000, 2000, model_alt, tokenizer, 4)

0.0 % complete
10.0 % complete
20.0 % complete
30.0 % complete
40.0 % complete
50.0 % complete
70.0 % complete
80.0 % complete
90.0 % complete


In [51]:
endf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          20000 non-null  object
 1   tweets      20000 non-null  object
 2   input       20000 non-null  object
 3   output      20000 non-null  object
 4   output_alt  20000 non-null  object
dtypes: object(5)
memory usage: 781.4+ KB


In [70]:
endf.head()

,id,tweets,input,output,output_alt,output_num,output_alt_num,output_mix_num_avg
0,35dad41e25a45dcf6d7e3dd905dd0914,#USER# #HASHTAG# are the #HASHTAG#!,user hashtag are the hashtag,hatespeach,hatespeach,1,1,0
1,35dad41e25a45dcf6d7e3dd905dd0914,#USER# Let's not pretend that you have a lick ...,user let us not pretend that you have a lick o...,hatespeach,neutral,1,0,0
2,35dad41e25a45dcf6d7e3dd905dd0914,#USER# No #HASHTAG# for any of these #HASHTAG#...,user no hashtag for any of these hashtag hashtag,hatespeach,hatespeach,1,1,0
3,35dad41e25a45dcf6d7e3dd905dd0914,Because you are against it - moving #HASHTAG# ...,because you are against it moving hashtag i...,neutral,neutral,0,0,0
4,35dad41e25a45dcf6d7e3dd905dd0914,When is #HASHTAG# going to #HASHTAG#? #HASHTAG...,when is hashtag going to hashtag hashtag hasht...,neutral,neutral,0,0,0


In [56]:
endf['output_num'] = endf['output'].parallel_apply(replace_bin)
endf['output_alt_num'] = endf['output_alt'].parallel_apply(replace_bin)

In [79]:
def assign_combined(df,step=200, weight=0.35):
    ids = []
    preds = []
    for i in range(0, int(df.shape[0]/step)):
        df.loc[i*step:(i+1)*step-1,'output_mix_num_avg'] = int(round(weight*df.loc[i*step:(i+1)*step-1,'output_alt_num'].mean()+(1-weight)*df.loc[i*step:(i+1)*step-1,'output_num'].mean()))
        ids.append(df.loc[i*step, 'id'])
        preds.append(df.loc[i*step, 'output_mix_num_avg'])
    for i in range(len(ids)):
        a_id = str(ids[i])
        pred = preds[i]   
        root = ET.Element("author", id =a_id, lang="en", type=str(pred))
        tree = ET.ElementTree(root)
        tree.write("results_en/"+ a_id + ".xml")
    return df

In [80]:
endf = assign_combined(endf)

In [81]:
endf.output_mix_num_avg = endf.output_mix_num_avg.astype(int)

In [82]:
endf.to_csv("results_en/results.csv")

In [83]:
endf.output_mix_num_avg.describe(percentiles = [0.25, 0.35,0.45, 0.55, 0.65])

count    20000.000000
mean         0.550000
std          0.497506
min          0.000000
25%          0.000000
35%          0.000000
45%          0.550000
50%          1.000000
55%          1.000000
65%          1.000000
max          1.000000
Name: output_mix_num_avg, dtype: float64

In [84]:
endf.output_mix_num_avg.value_counts()

1    11000
0     9000
Name: output_mix_num_avg, dtype: int64

In [78]:
esdf.output_num_avg.value_counts()

1    14800
0     5200
Name: output_num_avg, dtype: int64

In [85]:
esdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              20000 non-null  object
 1   tweets          20000 non-null  object
 2   input           20000 non-null  object
 3   output          20000 non-null  object
 4   output_num      20000 non-null  int64 
 5   output_num_avg  20000 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 937.6+ KB


In [98]:
freq_es = pd.crosstab(esdf.output_num, esdf.id).T

In [109]:
median_0 = freq_es[0].describe()['50%']

In [127]:
freq_es['result'] = 0

In [137]:
freq_es.head(15)

output_num,0,1,result
id,,,
03263875c28c5e589409ab4ee1d18c2f,169,31,0
0373317cba1543d5976c3a2cc2f92a79,35,165,1
03872a57faf2cf7eb3327e49b27d53bf,137,63,0
0389ca1de412788fe6e2ad1aff9a37b2,25,175,1
03b9385278a1ec7bdf7e5775c138d7a0,46,154,0
04101a3c7a11a159e965fdeb048e9386,20,180,1
0794ff63d194999ef7dfbb27fb67830c,24,176,1
0d1a59776b24ea8e8a924ddabf09ab14,40,160,1
15f18af6d4b4008f87be4bdc68898c23,31,169,1


In [131]:
freq_es.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 03263875c28c5e589409ab4ee1d18c2f to ff231322f3dacfd3fb63a5ed4013d484
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       100 non-null    int64
 1   1       100 non-null    int64
 2   result  100 non-null    int64
dtypes: int64(3)
memory usage: 7.2+ KB


In [134]:
def get_author_output(df, delimeter):
    for i in range(0, int(df.shape[0])):
        if df.iloc[i,0] >= delimeter:
            df.iloc[i,2] = 0
        else:
            df.iloc[i,2] = 1
    return df

In [135]:
freq_es = get_author_output(freq_es, median_0)

In [136]:
freq_es.result.value_counts()

0    50
1    50
Name: result, dtype: int64

In [139]:
for i in range(0, int(freq_es.shape[0])):
    a_id = str(freq_es.index[i])
    pred = freq_es.iloc[i,2]
    print(a_id, pred)
    root = ET.Element("author", id =a_id, lang="es", type=str(pred))
    tree = ET.ElementTree(root)
    tree.write("results_es/"+ a_id + ".xml")

03263875c28c5e589409ab4ee1d18c2f 0
0373317cba1543d5976c3a2cc2f92a79 1
03872a57faf2cf7eb3327e49b27d53bf 0
0389ca1de412788fe6e2ad1aff9a37b2 1
03b9385278a1ec7bdf7e5775c138d7a0 0
04101a3c7a11a159e965fdeb048e9386 1
0794ff63d194999ef7dfbb27fb67830c 1
0d1a59776b24ea8e8a924ddabf09ab14 1
15f18af6d4b4008f87be4bdc68898c23 1
17abfb488b6ca1cfc52310709d811de1 0
18172d25b495577ee01fdf2101dcb3cd 0
19f3683f4f99dfa0de0e4e937e7e3f28 0
1a08dc0ce59a97afaa09cb75fec56b7f 1
1b95d9df96f463fb405d9718da1798fe 0
1d6b3543165c1e22b7db4ecfb1fff4a4 1
1e8e213913f796e4114f2422fb97c532 0
26add9f0d7818961f984f889fb1d1f3f 0
290a1bab7de4d014773e389b37375586 1
2e57218e7912c49c1e68d508c5fb1eda 0
2ef652b2e2f47ed37813af4eead72f2d 0
305ff3d0cc125cd9552b8ff1aad5df46 0
335869e32becfc88c7d85eefa141540f 1
34aea5b60fdc422a087818eb931efaee 1
356e6d8242cbd06f0522a9b048ca9fe3 0
38dcd0a9fa868600da9a09eb3026848e 0
3933fd031f1f8ccfc7f42dbec07cb65d 1
3ba4d15387188355e0869fcab5b8f0d4 1
3d6535ead17b2653c70059e7b7d5c2dd 1
40df1901733dcb1bcbc0